# Convert Polar Stereographic Files to Lat/Lon Files

This notebook provides a workflow for converting polar stereographic netcdf files to lat/lon coordinates. To get the data into the proper format, we needed to do the following:
 - utilize a template file as a base for modifying netcdf files of the existing mask arrays. The template file includes all proper GIS attributes.
 - python script reads the Geotiff file and creates a netcdf file with the x,y,lat,lon coordinate dimensions and variables

Scripts were originally created by John Truesdale and updated by Teagan King.

Some notes from John on methods for regridding:
-----------------------------------------------
 - used QGIS to assign a standard stereographic coordinate and create a GeoTiff file version of one of the Polar jpg plots
 - Giving a standard location to each jpeg pixel basically consisted of finding similar features at the pixel level (the tip of an island, the most inset part of a promenent bay) between our polar jpegs and a map that is already georeferenced and has known locations for those pixels. If you choose 3-10 pixels in common you can create a linear regression that will map out all the rest of the pixels on your jpeg.
 - Once the GIS application was able to calculate the transform to go from pixel to a standard coordinate system, I saved all that information in a GeoTiff file.
 - The polar projection jpegs on which the climatenet masks are drawn were created from python matplotlib and you can grab the coordinate information from matplotlib; this was checked with QGIS
 - Because the LLNL polar jpegs are a projected coordinate, the underlying unit in a stereographic projection is meters.  The x and y variables on the GeoTiff and converted netcdf file contain meter offsets of every pixel (row,col) of the ar_mask array with respect to one of the standard south pole stereographic coordinate systems.
 -   When you look at the square projected polar image you see that the longitude lines converge at the pole and latitudes are a set of nested circles.  When you are describing this grid in lat/lon coordinates it is known as a curvilinear grid where each pixel (array location) requires a unique lat/lon pair to specify its position on a regular grid. A straight line along any row or column of the jpeg raster (or ar_mask array) will intersect different lat lon values for every pixel.  The coordinate information for our rectangular ar_mask array therefore contains lat and lon variables that are two dimensions and describe the entire grid of 1152x1152 points with unique lat/lon values for each pixel (array location) of ar_mask. The standard netcdf way of denoting a curvilinear coordinate is by creating the dimensions that define the size of the ar_mask array (x,y), adding lat/lon variables that are each dimensioned (x,y) containing the lat/lon coordinates of that point, and finally adding metadata to the ar_mask array noting that the coordinates for this variable are not the dimension variables x,y but the lat/lon variables.

Generate remapped IVT/TMQ/etc underlying data:  
 - ESMF_RegridWeightGen --ignore_unmapped --dst_regional -m bilinear -w map_fv0.23x0.31_to_sp_stereo_near.nc -s /glade/p/cesmdata/inputdata/share/scripgrids/fv0.23x0.31_141008.nc -d /glade/u/home/jet/sp_stereographic_SCRIP.nc  
 - ncremap -m ./map_fv0.23x0.31_to_sp_stereo_near.nc -i windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001-200012.nc -o polar_ivt/windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001-200012.nc  
 - see /glade/scratch/tking/cgnet/high_lat_QC/

In [1]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import os
from rasterio.warp import transform
import urllib.request
import xarray as xr
import glob
from netCDF4 import date2num
from netCDF4 import Dataset
import numpy as np
import datetime as dt
import cftime

### Generate template file (only needs to be done once)

In [2]:
# # Read the data
# input_path = '/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-copy-sav1.tif'

# da = xr.open_rasterio(input_path)
# yval=da['y']
# ryval=np.flip(yval)
# # Compute the lon/lat coordinates with rasterio.warp.transform
# ny, nx = len(da['y']), len(da['x'])
# x, y = np.meshgrid(da['x'], ryval)
# # Rasterio works with 1D arrays
# lon, lat = transform(da.crs, {'init': 'EPSG:4326'},
#                      x.flatten(), y.flatten())
# lon = np.asarray(lon).reshape((ny, nx))
# lat = np.asarray(lat).reshape((ny, nx))
# da.coords['lon'] = (('y', 'x'), lon)
# da.coords['lat'] = (('y', 'x'), lat)

In [3]:
# da.to_netcdf(path='/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-sav1-rev-latlon.nc')
# # use for just antarctic

### Change to 2-dimensional lat/lon before regridding 

In [4]:
# Import dask
import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
nmem='25GB' # specify memory here so it duplicates below
cluster = PBSCluster(
    cores=1, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='/glade/scratch/$USER/local_dask', # Use your local directory
    resource_spec='select=1:ncpus=1:mem='+nmem, # Specify resources
    account='P93300313', # Input your project ID here, previously this was known as 'project', now is 'account'
    walltime='04:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
cluster.scale(20)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.60:43217,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# define dictionary of file names

In [6]:
tmq_dict = {2000: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100.nc",
            2001: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200101010000-200112312100.nc",
            2002: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200201010000-200212312100.nc",
            2003: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200301010000-200312312100.nc",
            2004: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200401010000-200412312100.nc"}

ivt_dict = {2000: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001-200012.nc",
            2001: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200101-200112.nc",
            2002: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200201-200212.nc",
            2003: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200301-200312.nc",
            2004: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200401-200412.nc"}

psl_dict = {2000: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100.nc",
            2001: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200101010000-200112312100.nc",
            2002: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200201010000-200212312100.nc",
            2003: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200301010000-200312312100.nc",
            2004: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200401010000-200412312100.nc"}

pr_dict = {2000: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312359.nc",
           2001: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200101010000-200112312359.nc",
           2002: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200201010000-200212312359.nc",
           2003: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200301010000-200312312359.nc",
           2004: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200401010000-200412312359.nc",
           2005: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200501010000-200512052359.nc"}

In [7]:
for var in ['pr']:
    if var=='pr':
        dictionary = pr_dict
    elif var=='psl':
        dictionary = psl_dict
    elif var=='ivt':
        dictionary = ivt_dict
    for year in [2000]: #,2001,2002,2003,2004]:
        ds_before_regrid = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/{}/{}'.format(var, dictionary[year]))
        ds_before_regrid
        # we want this to be two dimensional lat/lon instead of 1

        # mesh is one useful tool but could do by hand
        # duplicate lat/lon array for lon/lat number of times
        # in order to have lat (y,x) and lon (y,x)

        # y is lat
        # x is lon
        # dimensions should be time, x, y, 
        # follow example here: https://xesmf.readthedocs.io/en/latest/notebooks/Curvilinear_grid.html
        y_len = ds_before_regrid.lon.shape[0]
        x_len = ds_before_regrid.lat.shape[0]

        ds_before_regrid['lat_val'] = (('y','x'), np.tile(ds_before_regrid.lat, (y_len,1)))
        ds_before_regrid['lon_val'] = (('y','x'), np.transpose(np.tile(ds_before_regrid.lon, (x_len,1))))

        # ds_before_regrid.to_netcdf('/glade/scratch/tking/cgnet/high_lat_QC/{}/2dlatlon/{}'.format(var,dictionary[year]))
        # print('generated {}'.format(dictionary[year]))
    

In [21]:
# ds_before_regrid

# MissingDimensionsError: 'lat' has more than 1-dimension and the same name as one of
# its dimensions ('lat', 'lon'). xarray disallows such variables because they conflict
# with the coordinates used to label dimensions.


In [19]:
ds_before_regrid['pr'] = ds_before_regrid.pr.swap_dims({'lat':'x','lon':'y'})

In [20]:
ds_before_regrid

<xarray.Dataset>
Dimensions:    (height: 1, lat: 768, lon: 1152, time: 2920, nb2: 2, x: 768,
                y: 1152)
Coordinates:
  * height     (height) float32 0.0
  * lat        (lat) float32 -90.0 -89.77 -89.53 -89.3 ... 89.3 89.53 89.77 90.0
  * lon        (lon) float32 0.0 0.3125 0.625 0.9375 ... 358.8 359.1 359.4 359.7
  * time       (time) object 2000-01-01 01:30:00 ... 2000-12-31 22:30:00
Dimensions without coordinates: nb2, x, y
Data variables:
    time_bnds  (time, nb2) object ...
    pr         (time, height, x, y) float32 ...
    lat_val    (y, x) float32 -90.0 -89.77 -89.53 -89.3 ... 89.53 89.77 90.0
    lon_val    (y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 359.7 359.7 359.7 359.7
Attributes: (12/22)
    history:                   2017-03-03T13:07:25Z: Naming and attribute con...
    forcing:                   SST=sst_HadOIBl_bc_0.23x0.31_1870_2008_c091020...
    institute_run_id:          cam5_1_amip_run2
    CDO:                       Climate Data Operators version 1.6.5.2 (http:/...
    institution:               Lawrence Berkeley National Laboratory, Berkele...
    institute_id:              LBNL
    ...                        ...
    parent_experiment:         N/A
    parent_subexperiment:      N/A
    parent_run_id:             N/A
    project_id:                C20C+ Detection and Attribution Project
    license:                   Creative Commons License: http://creativecommo...
    creation_date:             2017-03-03T13:07:25Z

In [28]:
ds_before_regrid = ds_before_regrid.drop_dims('lat')
ds_before_regrid = ds_before_regrid.drop_dims('lon')
ds_before_regrid

<xarray.Dataset>
Dimensions:    (height: 1, time: 2920, nb2: 2, x: 768, y: 1152)
Coordinates:
  * height     (height) float32 0.0
  * time       (time) object 2000-01-01 01:30:00 ... 2000-12-31 22:30:00
Dimensions without coordinates: nb2, x, y
Data variables:
    time_bnds  (time, nb2) object ...
    pr         (time, height, x, y) float32 ...
    lat_val    (y, x) float32 -90.0 -89.77 -89.53 -89.3 ... 89.53 89.77 90.0
    lon_val    (y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 359.7 359.7 359.7 359.7
Attributes: (12/22)
    history:                   2017-03-03T13:07:25Z: Naming and attribute con...
    forcing:                   SST=sst_HadOIBl_bc_0.23x0.31_1870_2008_c091020...
    institute_run_id:          cam5_1_amip_run2
    CDO:                       Climate Data Operators version 1.6.5.2 (http:/...
    institution:               Lawrence Berkeley National Laboratory, Berkele...
    institute_id:              LBNL
    ...                        ...
    parent_experiment:         N/A
    parent_subexperiment:      N/A
    parent_run_id:             N/A
    project_id:                C20C+ Detection and Attribution Project
    license:                   Creative Commons License: http://creativecommo...
    creation_date:             2017-03-03T13:07:25Z

In [29]:
ds_before_regrid.to_netcdf('/glade/scratch/tking/cgnet/high_lat_QC/{}/2dlatlon/{}'.format(var,dictionary[year]))

In [35]:
# regrid w/ batch script between these steps...

# Rename regridded files to avoid lat/lon being dimension and variable name

In [4]:
# use ncrename to rename the output lat/lon dims to not conflict with vars
# ncrename -v lon,longitude -d lat,latitude prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100.nc renamed/prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100.nc

pr_ds = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/prw/2dlatlon/polar_tmq/renamed/renamed_2/prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100.nc')

In [5]:
pr_ds

<xarray.Dataset>
Dimensions:    (latitude: 1152, longitude: 1152, nvertices: 4, time: 2919,
                nbnd: 2)
Coordinates:
  * time       (time) object 2000-01-01 03:00:00 ... 2000-12-31 21:00:00
Dimensions without coordinates: latitude, longitude, nvertices, nbnd
Data variables:
    lat        (latitude, longitude) float64 ...
    lon        (latitude, longitude) float64 ...
    area       (latitude, longitude) float64 ...
    lat_bnds   (latitude, longitude, nvertices) float64 ...
    lon_bnds   (latitude, longitude, nvertices) float64 ...
    prw        (time, latitude, longitude) float32 ...
    time_bnds  (time, nbnd) object ...
Attributes: (12/28)
    forcing:                   SST=sst_HadOIBl_bc_0.23x0.31_1870_2008_c091020...
    institute_run_id:          cam5_1_amip_run2
    institution:               Lawrence Berkeley National Laboratory, Berkele...
    institute_id:              LBNL
    experiment_family:         All-Hist
    experiment:                est1
    ...                        ...
    remap_hostname:            crhtc04
    remap_version:             4.7.9
    NCO:                       netCDF Operators version 5.0.3 (Homepage = htt...
    map_file:                  ../../map_fv0.23x0.31_to_sp_stereo_near.nc
    input_file:                /glade/scratch/tking/cgnet/high_lat_QC/prw/2dl...
    history:                   Mon Mar 27 11:34:43 2023: ncrename -d lat,lati...

## regrid TMQ/IVT/PSL/PR data from lat/lon to polar

In [3]:
# submit scripts below in batch scripts, see example at /glade/scratch/tking/cgnet/high_lat_QC/prw

In [314]:
# module load gnu/9.1.0
# module load esmf_libs/8.0.0
# module load esmf-8.0.0-ncdfio-mpi-O
# module load nco/4.7.9

In [315]:
# set srcgrid=f09
# set dstgrid=sp_stereo
# set srcgridfile=/glade/p/cesmdata/inputdata/share/scripgrids/fv0.23x0.31_141008.nc
# set dstgridfile=/glade/u/home/jet/sp_stereographic_SCRIP.nc
# set srcinitfile=prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100.nc
# set dstinitfile=polar_tmq/prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100_polar_nearest.nc

# #create the map file
# ESMF_RegridWeightGen --ignore_unmapped --src_regional -m neareststod -w map_${srcgrid}_to_${dstgrid}_near.nc -s ${srcgridfile} -d ${dstgridfile}

# #use the mapfile to remap srcinitfile to dstinitfile
# ncremap -m ./map_${srcgrid}_to_${dstgrid}_near.nc -i ${srcinitfile} -o ${dstinitfile}


# currently running batch scripts on Thursday March 16th ~1:20pm!

## loop through mask files to generate converted mask files

In [312]:
# for year in [2001]:  # , 2002, 2003]:
#     qa_antarctic = sorted(glob.glob('/glade/u/home/tking/work/cgnet/QA_xml/h5/qa*/antarctic/netcdfs/data-{}*'.format(year)))

#     tmq = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/prw/{}'.format(tmq_dict[year]))
#     # lat: 768 lon: 1152 time: 2919 nbnd: 2
#     psl = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/psl/{}'.format(psl_dict[year]))
#     # lat: 768 lon: 1152 time: 2919 nbnd: 2
#     ivt = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/ivt/{}'.format(ivt_dict[year]))
#     # time: 2920 bound: 2 lat: 768 lon: 1152
#     pr = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/pr/{}'.format(pr_dict[year]))
#     # height: 1 lat: 768 lon: 1152 time: 2920 nb2: 2
#     # these have different coords (although tmq and psl are same, ivt just seems to be reordered/renamed,
#     # pr has an extra height dimension (that is constant) and renamed other dims)
#     time_index = 0
#     for index in range(len(qa_antarctic[:3])):

#         # read in mask file
#         qa_aa_ds = xr.open_dataset(qa_antarctic[index])

#         # read in temp file for use in notebook
#         temp = xr.open_dataset('/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-sav1-rev-latlon.nc')

#         # rename the dimensions of the original ar_masks variable from phony_0/1 to y and x respectively
#         qa_aa_ds = qa_aa_ds.rename({'phony_dim_0':'y','phony_dim_1':'x'})

#         time = qa_antarctic[index].split('/')[-1].split('data-')[1].split('.nc')[0].split('-00-2')[0]
#         time_year = int(time.split('-')[0])  # or = year
#         time_month = int(time.split('-')[1])
#         time_day = int(time.split('-')[2])
#         time_hour = 0  # all files were 00

#         # determine hours past a base date

#         file_for_date = qa_antarctic[index].split('/')[-1].split('data-')[1].split('.nc')[0].split('-00-2')[-1]
#         if file_for_date == '':
#             sample_id = 0
#         else:
#             sample_id = int(file_for_date[1]) # format would be _#, want #
#         if sample_id == 0:
#             time_index += 1

#         # todo: make this temp.time[time_index]...
#         date_number = date2num(dt.datetime(time_year, time_month, time_day, time_hour, 0), 'hours since 1970-01-01')
#         temp['time'] = date_number
#         temp['time'] = temp['time'].assign_attrs({'long_name': 'time',
#                                                   'units': 'hours since 1970-01-01',
#                                                   'calendar': 'noleap'})


#         # append the new x,y,lat,lon variables to mask using the GeoTiff temp file
#         temp['ar_masks'] = qa_aa_ds.ar_masks

#         temp['ar_masks'] = temp['ar_masks'].assign_coords({'sample_id': sample_id})
#         # temp['ar_masks'] = temp['ar_masks'].assign_coords({'time': date_number})
        
#         # add all the coordinate attributes to ar_mask
#         temp['ar_masks'] = temp['ar_masks'].assign_attrs({'transform' : temp.__xarray_dataarray_variable__.transform,
#                                                           'crs': temp.__xarray_dataarray_variable__.crs,
#                                                           'res': temp.__xarray_dataarray_variable__.res,
#                                                           'is_tiled': temp.__xarray_dataarray_variable__.is_tiled,
#                                                           'nodatavals': temp.__xarray_dataarray_variable__.nodatavals,
#                                                           'scales': temp.__xarray_dataarray_variable__.scales,
#                                                           'offsets': temp.__xarray_dataarray_variable__.offsets,
#                                                           'AREA_OR_POINT': temp.__xarray_dataarray_variable__.AREA_OR_POINT})
        
#         if time_month < 10:
#             time_m_formatted = '0'+str(time_month)
#         else:
#             time_m_formatted = str(time_month)
#         if time_day < 10:
#             time_d_formatted = '0'+str(time_day)
#         else:
#             time_d_formatted = str(time_day)
            
#         temp['date'] = str(time_year)+time_m_formatted+time_d_formatted
#         temp['date'] = temp['date'].assign_attrs({'long_name':'current date (YYYYMMDD)'})
#         temp['datesec'] = str(sample_id)
#         temp['datesec'] = temp['datesec'].assign_attrs({'long_name':'current seconds of current date'})

#         # put in all the variable data that we received from Sol (TMQ, IVT, PSL, etc.)
#         # These variables will be dimensioned (time,y,x) in your single output file and
#         # have the same lat lon coordinate attribute as the ar_mask array.
#         # construct the filenames for TMQ, IVT etc from the date info that you have
#         # then open, read and close those files inside your processing loop.

#         # open tmq file for the correct date and read tmq 
#         tmq_subset = tmq.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest')
#         psl_subset = psl.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest')
#         ivt_subset = ivt.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest')
#         pr_subset = pr.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest')
    
#         temp['tmq'] = tmq_subset.drop('time_bnds').drop('time').to_array()[0]
#         temp['psl'] = psl_subset.drop('time_bnds').drop('time').to_array()[0]
#         temp['ivt'] = ivt_subset.drop('time').drop('time_bnds').drop('bounds_lat').drop_dims('bound').to_array()[0]
#         temp['pr'] = pr_subset.drop('time_bnds').drop('time').to_array()[0][0]
        
#         temp['tmq'] = temp['tmq'].assign_coords({'sample_id': sample_id})
#         temp['psl'] = temp['psl'].assign_coords({'sample_id': sample_id})
#         temp['ivt'] = temp['ivt'].assign_coords({'sample_id': sample_id})
#         temp['pr'] = temp['pr'].assign_coords({'sample_id': sample_id})
        
#         # todo: resolve issue with repeated dimension
#         # temp['tmq'] = temp['tmq'].assign_coords({'time': date_number})
#         # temp['psl'] = temp['psl'].assign_coords({'time': date_number})
#         # temp['ivt'] = temp['ivt'].assign_coords({'time': date_number})
#         # temp['pr'] = temp['pr'].assign_coords({'time': date_number})
        
#         #tmq values seem to be present, other vars are missing values
#         temp['tmq'] = temp['tmq'].assign_attrs({'crs': temp['__xarray_dataarray_variable__'].crs,
#                                                 'transform': temp['__xarray_dataarray_variable__'].transform, 
#                                                 'res':temp['__xarray_dataarray_variable__'].res,
#                                                 'is_tiled':temp['__xarray_dataarray_variable__'].is_tiled,
#                                                 'nodatavals':temp['__xarray_dataarray_variable__'].nodatavals,
#                                                 'scales':temp['__xarray_dataarray_variable__'].scales,
#                                                 'offsets': temp['__xarray_dataarray_variable__'].offsets,
#                                                 'AREA_OR_POINT': temp['__xarray_dataarray_variable__'].AREA_OR_POINT})
#         temp['psl'] = temp['psl'].assign_attrs({'crs': temp['__xarray_dataarray_variable__'].crs,
#                                                 'transform': temp['__xarray_dataarray_variable__'].transform, 
#                                                 'res':temp['__xarray_dataarray_variable__'].res,
#                                                 'is_tiled':temp['__xarray_dataarray_variable__'].is_tiled,
#                                                 'nodatavals':temp['__xarray_dataarray_variable__'].nodatavals,
#                                                 'scales':temp['__xarray_dataarray_variable__'].scales,
#                                                 'offsets': temp['__xarray_dataarray_variable__'].offsets,
#                                                 'AREA_OR_POINT': temp['__xarray_dataarray_variable__'].AREA_OR_POINT})
#         temp['pr'] = temp['pr'].assign_attrs({'crs': temp['__xarray_dataarray_variable__'].crs,
#                                                 'transform': temp['__xarray_dataarray_variable__'].transform, 
#                                                 'res':temp['__xarray_dataarray_variable__'].res,
#                                                 'is_tiled':temp['__xarray_dataarray_variable__'].is_tiled,
#                                                 'nodatavals':temp['__xarray_dataarray_variable__'].nodatavals,
#                                                 'scales':temp['__xarray_dataarray_variable__'].scales,
#                                                 'offsets': temp['__xarray_dataarray_variable__'].offsets,
#                                                 'AREA_OR_POINT': temp['__xarray_dataarray_variable__'].AREA_OR_POINT})
#         temp['ivt'] = temp['ivt'].assign_attrs({'crs': temp['__xarray_dataarray_variable__'].crs,
#                                                 'transform': temp['__xarray_dataarray_variable__'].transform, 
#                                                 'res':temp['__xarray_dataarray_variable__'].res,
#                                                 'is_tiled':temp['__xarray_dataarray_variable__'].is_tiled,
#                                                 'nodatavals':temp['__xarray_dataarray_variable__'].nodatavals,
#                                                 'scales':temp['__xarray_dataarray_variable__'].scales,
#                                                 'offsets': temp['__xarray_dataarray_variable__'].offsets,
#                                                 'AREA_OR_POINT': temp['__xarray_dataarray_variable__'].AREA_OR_POINT})
#         temp = temp.drop_vars('__xarray_dataarray_variable__')
#         temp = temp.drop_dims('band')
#         temp = temp.drop_vars('variable')
#         # temp = temp.drop('band') # drop index, as well
#         filename = qa_antarctic[index].split('/')[-1]
#         temp.to_netcdf(path='/glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/'+filename)
#         print('made netcdf {}'.format(filename))

#         qa_aa_ds.close()

#     tmq.close()
#     ivt.close()
#     psl.close()
#     pr.close()

made netcdf data-2001-01-01-00-2_0.nc
made netcdf data-2001-01-24-00-2_0.nc
made netcdf data-2001-01-24-00-2_1.nc


In [311]:
# this seems to work, but turns to all nan's when set temp['ivt'] equal to it...
# ivt_temp = ivt.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest').drop('time').drop('time_bnds').drop('bounds_lat').drop('bounds_lon').to_array()[0]
# temp['ivt'] = ivt_temp

# print(temp['ivt'])
# print(ivt_temp)

## try using different setup; create netcdf from scratch

In [7]:
rm /glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/temp.nc

In [8]:
temp_file = '/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-sav1-rev-latlon.nc'

ncfile = Dataset('/glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/temp.nc',mode='w',format='NETCDF4_CLASSIC') 

lat_dim = ncfile.createDimension('lat', 1152)     # latitude axis
lon_dim = ncfile.createDimension('lon', 1152)    # longitude axis
y_dim = ncfile.createDimension('y', 1152)        # latitude axis
x_dim = ncfile.createDimension('x', 1152)        # longitude axis
time_dim = ncfile.createDimension('time', None)  # unlimited axis (can be appended to)
# time_index_dim = ncfile.createDimension('time_index', 350)
sample_id_dim = ncfile.createDimension('sample_id', 4)

# include time variable and relevant attributes
time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'hours since 1970-01-01'
time.calendar = 'noleap'
time.long_name = 'time'

date = ncfile.createVariable('date', np.float64, ('time',))
date.long_name = 'current date (YYYYMMDD)'
datesec = ncfile.createVariable('datesec', np.float64, ('time',))
date.long_name = 'current seconds of current date'

# include ar_mask variable and relevant attributes
ar_mask = ncfile.createVariable('ar_mask', np.float64, ('time','sample_id','y','x'))
ar_mask.units = 'hours since 1970-01-01'
ar_mask.calendar = 'noleap'
ar_mask.long_name = 'time'

# include underlying data
tmq = ncfile.createVariable('tmq',np.float64,('lat','lon','sample_id','time'))
ivt = ncfile.createVariable('ivt',np.float64,('lat','lon','sample_id','time'))
psl = ncfile.createVariable('psl',np.float64,('lat','lon','sample_id','time'))
pr = ncfile.createVariable('pr',np.float64,('lat','lon','sample_id','time'))

time_index = 0
for year in [2001]: # 2002, 2003
    mask_file_list = sorted(glob.glob('/glade/u/home/tking/work/cgnet/QA_xml/h5/qa*/antarctic/netcdfs/data-{}*'.format(year)))
    
    tmq_ds = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/prw/2dlatlon/polar_tmq/renamed/{}'.format(tmq_dict[year]))
    # lat: 768 lon: 1152 time: 2919 nbnd: 2
    psl_ds = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/psl/2dlatlon/polar_psl/renamed/{}'.format(psl_dict[year]))
    # lat: 768 lon: 1152 time: 2919 nbnd: 2
    ivt_ds = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/ivt/2dlatlon/polar_ivt/renamed/{}'.format(ivt_dict[year]))
    # time: 2920 bound: 2 lat: 768 lon: 1152
    pr_ds = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/pr/2dlatlon/polar_pr/renamed/{}'.format(pr_dict[year]))
    # height: 1 lat: 768 lon: 1152 time: 2920 nb2: 2
    # todo: use polar subdirectories to get regridded data!!!
    for antarctic_mask_file in mask_file_list[:1]:
        # try: ncfile.close()  # just to be safe, make sure dataset is not already open.
        # except: pass

        time = antarctic_mask_file.split('/')[-1].split('data-')[1].split('.nc')[0].split('-00-2')[0]
        time_year = int(time.split('-')[0])  # or = year
        time_month = int(time.split('-')[1])
        time_day = int(time.split('-')[2])
        time_hour = 0  # all files were 00
        date_number = date2num(dt.datetime(time_year, time_month, time_day, time_hour, 0), 'hours since 1970-01-01')
        
        if time_month < 10:
            time_m_formatted = '0'+str(time_month)
        else:
            time_m_formatted = str(time_month)
        if time_day < 10:
            time_d_formatted = '0'+str(time_day)
        else:
            time_d_formatted = str(time_day)

        sample_id = antarctic_mask_file[-4:-3]
        if sample_id == '0':
            time_index+=1
        
        qa_aa_ds = xr.open_dataset(antarctic_mask_file)

        # fill in date and datesec
        date[:] = str(time_year)+time_m_formatted+time_d_formatted
        datesec[:] = '00'

        # fill in ar_mask
        ar_mask[time_index,sample_id,:,:]=qa_aa_ds.ar_masks

        # open tmq file for the correct date and read tmq 
        tmq_subset = tmq_ds.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 3, 0, 0, 0, has_year_zero=True), method='nearest')
        psl_subset = psl_ds.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 3, 0, 0, 0, has_year_zero=True), method='nearest')
        # ivt_subset = ivt_ds.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 3, 0, 0, 0, has_year_zero=True), method='nearest')
        pr_subset = pr_ds.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 3, 0, 0, 0, has_year_zero=True), method='nearest')

        tmq_subset_array = tmq_subset.to_array()
        tmq_subset_array['time'] = 200101013000
        # todo: maybe look at https://cf-xarray.readthedocs.io/en/latest/
        tmq[:,:,sample_id,time_index] = tmq_subset_array.isel(variable=5).isel(nvertices=0,nbnd=0)
        # todo: is selecting nvertices=0, etc ok?
        
        psl_subset_array = psl_subset.to_array()
        psl_subset_array['time'] = 200101013000
        psl[:,:,sample_id,time_index] = psl_subset_array.isel(variable=5).isel(nvertices=0,nbnd=0)
        
        pr_subset_array = pr_subset.to_array()
        pr_subset_array['time'] = 200101013000
        print('before adding')
        print(pr[:,:,sample_id,time_index])
        pr[:,:,sample_id,time_index] = pr_subset_array.isel(variable=5).isel(nvertices=0, height=0, nb2=0)
        print('after adding')
        print(pr[:,:,sample_id,time_index])
        
        print(ncfile)
        
    # todo: write netcdf here
# or here

before adding
[[-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 ...
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]]
after adding
[[8.81081449e-13 1.00563731e-12 1.10457054e-12 ... 6.57584242e-05
  6.38374331e-05 6.29504793e-05]
 [9.78930695e-13 1.10360527e-12 1.17117557e-12 ... 6.41366860e-05
  6.26789842e-05 6.17809928e-05]
 [1.13347190e-12 1.18265478e-12 1.24873213e-12 ... 6.25432731e-05
  6.10857751e-05 5.97660182e-05]
 ...
 [2.25768217e-05 2.45883948e-05 2.64726459e-05 ... 1.04061570e-09
  7.54678382e-08 1.17062626e-07]
 [2.48554279e-05 2.69527754e-05 2.92104851e-05 ... 5.79620885e-09
  2.06496931e-09 1.55101425e-08]
 [2.69287357e-05 2.97736842e-05 3.20320505e-05 ... 1.33372025e-08
  9.59605728e-09 2.56190158e-09]]
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): lat(1152), lon(1152), y(1152), x(1152), time(2), sample_id(4)
    variables(dimensions): float64 tim

In [9]:
ncfile['pr']

<class 'netCDF4._netCDF4.Variable'>
float64 pr(lat, lon, sample_id, time)
unlimited dimensions: time
current shape = (1152, 1152, 4, 2)
filling on, default _FillValue of 9.969209968386869e+36 used

In [10]:
pr

<class 'netCDF4._netCDF4.Variable'>
float64 pr(lat, lon, sample_id, time)
unlimited dimensions: time
current shape = (1152, 1152, 4, 2)
filling on, default _FillValue of 9.969209968386869e+36 used

In [11]:
pr_subset_array.isel(variable=5).isel(nvertices=0, height=0, nb2=0)

<xarray.DataArray (latitude: 1152, longitude: 1152)>
array([[8.810814491194696e-13, 1.0056373052000356e-12,
        1.1045705365989034e-12, ..., 6.575842417078093e-05,
        6.383743311744183e-05, 6.295047933235765e-05],
       [9.789306951862864e-13, 1.1036052714047395e-12,
        1.1711755697196846e-12, ..., 6.413668597815558e-05,
        6.2678984249942e-05, 6.17809928371571e-05],
       [1.133471897070415e-12, 1.1826547770613094e-12,
        1.248732128984742e-12, ..., 6.254327308852226e-05,
        6.108577508712187e-05, 5.97660182393156e-05],
       ...,
       [2.2576821720576845e-05, 2.4588394808233716e-05,
        2.647264591359999e-05, ..., 1.0406157047171405e-09,
        7.546783820089331e-08, 1.1706262625921227e-07],
       [2.4855427909642458e-05, 2.6952775442623533e-05,
        2.921048508142121e-05, ..., 5.796208846220452e-09,
        2.0649693066587815e-09, 1.5510142503671887e-08],
       [2.692873567866627e-05, 2.9773684218525887e-05,
        3.2032050512498245e-05, ..., 1.3337202453556074e-08,
        9.596057282124093e-09, 2.561901579767323e-09]], dtype=object)
Coordinates:
    height    float32 0.0
    time      int64 200101013000
    variable  <U2 'pr'
Dimensions without coordinates: latitude, longitude
Attributes: (12/29)
    forcing:                   SST=sst_HadOIBl_bc_0.23x0.31_1870_2008_c091020...
    institute_run_id:          cam5_1_amip_run2
    CDO:                       Climate Data Operators version 1.6.5.2 (http:/...
    institution:               Lawrence Berkeley National Laboratory, Berkele...
    institute_id:              LBNL
    experiment_family:         All-Hist
    ...                        ...
    remap_hostname:            crhtc28
    remap_version:             4.7.9
    NCO:                       netCDF Operators version 5.0.3 (Homepage = htt...
    map_file:                  ../../map_fv0.23x0.31_to_sp_stereo_near.nc
    input_file:                /glade/scratch/tking/cgnet/high_lat_QC/pr/2dla...
    history:                   Mon Mar 27 14:54:47 2023: ncrename -v lon,long...

In [12]:
pr[:,:,sample_id,time_index] = pr_subset_array.isel(variable=5).isel(nvertices=0, height=0, nb2=0)

In [13]:
pr

<class 'netCDF4._netCDF4.Variable'>
float64 pr(lat, lon, sample_id, time)
unlimited dimensions: time
current shape = (1152, 1152, 4, 2)
filling on, default _FillValue of 9.969209968386869e+36 used

In [14]:
ncfile.close()

In [171]:
type(pr_subset_array.isel(variable=5).isel(nvertices=0, height=0, nb2=0))

xarray.core.dataarray.DataArray

In [106]:
tmq_ds.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 3, 0, 0, 0, has_year_zero=True))

<xarray.Dataset>
Dimensions:    (latitude: 1152, longitude: 1152, nvertices: 4, nbnd: 2)
Coordinates:
    time       object 2001-01-01 03:00:00
Dimensions without coordinates: latitude, longitude, nvertices, nbnd
Data variables:
    lat        (latitude, longitude) float64 -0.4845 -0.5344 ... -0.6127 -0.5626
    lon        (latitude, longitude) float64 -135.1 -135.1 ... 44.93 44.98
    area       (latitude, longitude) float64 1.52e-06 1.521e-06 ... 1.524e-06
    lat_bnds   (latitude, longitude, nvertices) float64 -0.4344 ... -0.5625
    lon_bnds   (latitude, longitude, nvertices) float64 -135.1 -135.1 ... 44.93
    prw        (latitude, longitude) float32 ...
    time_bnds  (nbnd) object ...
Attributes: (12/28)
    forcing:                   SST=sst_HadOIBl_bc_0.23x0.31_1870_2008_c091020...
    institute_run_id:          cam5_1_amip_run2
    institution:               Lawrence Berkeley National Laboratory, Berkele...
    institute_id:              LBNL
    experiment_family:         All-Hist
    experiment:                est1
    ...                        ...
    remap_hostname:            crhtc04
    remap_version:             4.7.9
    NCO:                       netCDF Operators version 5.0.3 (Homepage = htt...
    map_file:                  ../../map_fv0.23x0.31_to_sp_stereo_near.nc
    input_file:                /glade/scratch/tking/cgnet/high_lat_QC/prw/2dl...
    history:                   Mon Mar 27 11:56:18 2023: ncrename -d lat,lati...

In [93]:
# tmq[:,:,sample_id,time_index] = tmq_ds.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 3, 0, 0, 0, has_year_zero=True)).to_array()[:6]

# tmq_subset_array = tmq_subset.to_array()
# tmq_subset_array['time'] = 200101013000
# tmq[:,:,sample_id,time_index] = tmq_subset_array.isel(variable=5).isel(nvertices=0,nbnd=0)

In [94]:
# tmq

In [100]:
# ncfile.to_netcdf('/glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/tmq_test.nc')
# tmq.to_netcdf('/glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/tmq_test.nc')

In [99]:
ncfile.close()
# this gets written anyways... although it doesn't seem to have expected data...

In [102]:
tmq_subset_array.isel(variable=5).isel(nvertices=0,nbnd=0).to_netcdf('/glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/tmq_test.nc')

# PART 2:

### use the standard ESMF mapping procedure to go from the projected stereographic polar mask grids to our regular gridded data from CESM. Use Steve's python code and John's mapping file commands to do this

In [35]:
# We want to do this to get masks and state information on two different grids
# need new mapping file to do this, then run through Steve's routine

In [ ]:
# Steve Yeager has a utility function for remapping CAM-SE output (see remap_camse function below):
#     https://github.com/sgyeager/mypyutils/blob/main/mypyutils/regrid_utils.py

import xarray as xr
import numpy as np
import scipy.sparse as sps
import cf_xarray

def remap_camse(ds, dsw, varlst=[]):
    #dso = xr.full_like(ds.drop_dims('ncol'), np.nan)
    dso = ds.drop_dims('ncol').copy()
    lonb = dsw.xc_b.values.reshape([dsw.dst_grid_dims[1].values, dsw.dst_grid_dims[0].values])
    latb = dsw.yc_b.values.reshape([dsw.dst_grid_dims[1].values, dsw.dst_grid_dims[0].values])
    weights = sps.coo_matrix((dsw.S, (dsw.row-1, dsw.col-1)), shape=[dsw.dims['n_b'], dsw.dims['n_a']])
    if not varlst:
        for varname in list(ds):
            if 'ncol' in(ds[varname].dims):
                varlst.append(varname)
        if 'lon' in varlst: varlst.remove('lon')
        if 'lat' in varlst: varlst.remove('lat')
        if 'area' in varlst: varlst.remove('area')
    for varname in varlst:
        shape = ds[varname].shape
        invar_flat = ds[varname].values.reshape(-1, shape[-1])
        remapped_flat = weights.dot(invar_flat.T).T
        remapped = remapped_flat.reshape([*shape[0:-1], dsw.dst_grid_dims[1].values,
                                          dsw.dst_grid_dims[0].values])
        dimlst = list(ds[varname].dims[0:-1])
        dims={}
        coords={}
        for it in dimlst:
            dims[it] = dso.dims[it]
            coords[it] = dso.coords[it]
        dims['lat'] = int(dsw.dst_grid_dims[1])
        dims['lon'] = int(dsw.dst_grid_dims[0])
        coords['lat'] = latb[:,0]
        coords['lon'] = lonb[0,:]
        remapped = xr.DataArray(remapped, coords=coords, dims=dims, attrs=ds[varname].attrs)
        dso = xr.merge([dso, remapped.to_dataset(name=varname)])
    return dso

In [ ]:
# Here is a notebook demonstrating how this is used:
#     /glade/u/home/yeager/analysis/python/toshare/CLM_field_regrid.ipynb